In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast # used to convert label column into python tuples
from itertools import chain # tool to create chains of text
from sklearn.preprocessing import MultiLabelBinarizer # used to binarize labels for easier processing
import torch.nn as nn # classes to help buld neural nets
import torch.nn.functional as F # gives stateless version of activation functions and nn layers
import cv2 
from matplotlib import pyplot as plt
from IPython.display import display
from torch.utils.data import Dataset
from torchvision import transforms

df = pd.read_csv('/kaggle/input/all-shapes-and-colors/train_v2.csv')

df['label'] = df['label'].apply(ast.literal_eval) # convert label to tuples safely

# create set of all labels
label_set = set(chain.from_iterable(df['label']))

# binarize lables and put into dataframe (multi-hot vector of classes)
mlb = MultiLabelBinarizer()
labels_binarized = mlb.fit_transform(df['label'])
labels_df = pd.DataFrame(labels_binarized, columns=mlb.classes_)
df = pd.concat([df, labels_df], axis=1)

print(df)


                         image_path  \
0        train_dataset_v2/img_0.png   
1        train_dataset_v2/img_1.png   
2        train_dataset_v2/img_2.png   
3        train_dataset_v2/img_3.png   
4        train_dataset_v2/img_4.png   
...                             ...   
4995  train_dataset_v2/img_4995.png   
4996  train_dataset_v2/img_4996.png   
4997  train_dataset_v2/img_4997.png   
4998  train_dataset_v2/img_4998.png   
4999  train_dataset_v2/img_4999.png   

                                                  label  (circle, blue)  \
0                                     [(square, green)]               0   
1                                                    []               0   
2                                                    []               0   
3     [(square, red), (circle, blue), (triangle, gre...               1   
4                                                    []               0   
...                                                 ...             ...   
4995  

In [7]:
# output will have N(6) neurones, one for each color-shape combo
# rgb & circle square only?

# use sigmoid activation function -> doesnt have to add up to 1 so allows for multiple classes to pass through better

class CNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__() # make sure nn.module is initialized
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(32 * 16 * 16, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # → (16, 32, 32)
        x = self.pool(F.relu(self.conv2(x)))  # → (32, 16, 16)
        x = x.view(x.size(0), -1)             # Flatten
        x = self.fc(x)       # output?
        return x

In [14]:
# custom dataset that helps load images from the csv
class ShapeColorDataset(Dataset):
    def __init__(self, df, image_root, class_names, transform=None):
        self.df = df
        self.image_root = image_root
        self.class_names = class_names
        self.transform = transform

    def __len__(self):
        return len(self.df)

    # a special type of pythonn function that tells the object how to behave when accessed with []
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = cv2.imread(f"{self.image_root}/{row['image_path']}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if self.transform:
            img = self.transform(img)

        label = torch.tensor(row[self.class_names].values.astype(np.float32)) # might already be doing thi multi hot conversion
        return img, label


In [18]:
# define transforms 
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((64,64)),
    transforms.ToTensor(),
])

dataset = ShapeColorDataset(df, image_root='/kaggle/input/all-shapes-and-colors/dataset_v2/train_dataset_v2', class_names=mlb.classes_, transform=transform)
dataloader = DataLoader(dataset, barch_size=32, shuffle=True) # idk

# create model, loss and optimizer
model = CNN(num_classes=len(mlb.classes_)) # makes class outputs modular
criterion = nn.BCEWithLogitsLoss() # write some stuff about why
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

NameError: name 'DataLoader' is not defined

In [ ]:
model.train() # where is this func
for epoch in range(5):
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")
